In [1]:
#For Google Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
%pip install numpy scipy
%pip install matplotlib
%pip install tensorflow
%pip install -U scikit-learn

In [1]:
import matplotlib
import sys
sys.path.append('/content/drive/MyDrive/Master_Project_3I/')

from model.ResNet50 import AgeEstimatorModel as ResNetModel
from model.EfficientNetB0 import AgeEstimatorModel1 as EfficientNetModel
from model.GoogleNet import AgeEstimatorModel2 as GoogleNetModel
from model.VGGNet import AgeEstimatorModel3 as VGGNetModel
from model.AlexNet import AlexNetAgeDetection as AlexNetModel
from model.CNNmodel import AgeEstimatorModel5 as CNNModel
import tensorflow as tf
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import glob

In [21]:
def train_model(model, model_name):
    # compile the model
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
    model.compile(loss="mean_squared_error", optimizer=opt, metrics=["mae"])  # using mean squared error loss

    # train the model
    H = model.fit(aug.flow(trainX, trainY, batch_size=batch_size),
                  validation_data=(testX, testY),
                  steps_per_epoch=len(trainX) // batch_size,
                  epochs=epochs, verbose=1)

    # save the model to disk
    model.save(f"{model_name}_{lr}_age_detection.model")

    return H

In [22]:
def plot_model_performance(H, model_name):
    # plot training/validation loss/mae
    plt.style.use("ggplot")
    plt.figure()
    N = epochs
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")

    plt.title(f"{model_name} Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="upper right")
    plt.savefig(f"{model_name}_loss_plot.png")

    plt.style.use("ggplot")
    plt.figure()
    N = epochs
    plt.plot(np.arange(0, N), H.history["mae"], label="train_mae")
    plt.plot(np.arange(0, N), H.history["val_mae"], label="val_mae")

    plt.title(f"{model_name} Training MAE")
    plt.xlabel("Epoch #")
    plt.ylabel("MAE")
    plt.legend(loc="upper right")
    plt.savefig(f"{model_name}_mae_plot.png")


In [45]:
# Paths and parameters initialization for Google Colab
dataset_path = "/content/drive/MyDrive/Master_Project_3I/Faces"
model_path_resnet = "/content/drive/MyDrive/Master_Project_3I/resnet_age_detection.model"
model_path_effnet = "/content/drive/MyDrive/Master_Project_3I/effnet_age_detection.model"
model_path_alexnet = "/content/drive/MyDrive/Master_Project_3I/AlexNet_age_detection.model"
model_path_vggnet = "/content/drive/MyDrive/Master_Project_3I/VGGNet_age_detection.model"
model_path_googlenet = "/content/drive/MyDrive/Master_Project_3I/googlenet_age_detection.model"
model_path_cnn = "/content/drive/MyDrive/Master_Project_3I/CNNModel_age_detection.model"
plot_path_resnet = "/content/drive/MyDrive/Master_Project_3I/resnet_plot.png"
plot_path_effnet = "/content/drive/MyDrive/Master_Project_3I/effnet_plot.png"
plot_path_googlenet = "/content/drive/MyDrive/Master_Project_3I/googlenet_plot.png"
plot_path_cnn = "/content/drive/MyDrive/Master_Project_3I/cnn_plot.png"
plot_path_alexnet = "/content/drive/MyDrive/Master_Project_3I/alexnet_plot.png"
plot_path_vggnet = "/content/drive/MyDrive/Master_Project_3I/vggnet_plot.png"

In [ ]:
# Paths and parameters initialization for local 
dataset_path = "Faces"
model_path_resnet = "resnet_age_detection.model"
model_path_effnet = "effnet_age_detection.model"
model_path_googlenet = "googlenet_age_detection.model"
model_path_vggnet = "vggnet_age_detection.model"
model_path_cnn = "cnn_age_detection.model"
model_path_alexnet = "alexnet_age_detection.model"
plot_path_resnet = "resnet_plot.png"
plot_path_effnet = "effnet_plot.png"
plot_path_googlenet = "googlenet_plot.png"
plot_path_alexnet = "alexnet_plot.png"
plot_path_vggnet = "vggnet_plot.png"
plot_path_cnn = "cnn_plot.png"
plot_path_alexnet2 = "alexnet2_plot.png"

In [43]:
# Initial parameters
epochs = 100
lr = 0.001
batch_size = 64
img_dims = (96, 96, 3)

# Load and preprocess the data
data = []
age_labels = []

# Load image files from the dataset
image_files = [f for f in glob.glob(dataset_path + "/*.jpg", recursive=True)]

print(len(image_files))

random.seed(42)
random.shuffle(image_files)

# Create ground-truth labels from the image paths
for img in image_files:
    image = cv2.imread(img)
    image = cv2.resize(image, (img_dims[0], img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    # Extract age labels from the image paths
    label = int(img.split("/")[-1].split("_")[0])
    age_labels.append(label)




4521


In [25]:
# Pre-process the data
data = np.array(data, dtype="float") / 255.0
labels = np.array(age_labels)

print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

# Augmenting dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


Data shape: (4521, 96, 96, 3)
Labels shape: (4521,)


In [ ]:
# build and train ResNet50 model
model_resnet = ResNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
H_resnet = train_model(model_resnet, "ResNet50")
plot_model_performance(H_resnet, "ResNet50")

In [ ]:
# build and train EfficientNetB0 model
model_effnet = EfficientNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
H_effnet = train_model(model_effnet, "EfficientNetB0")
plot_model_performance(H_effnet, "EfficientNetB0")

In [ ]:
# build and train GoogleNet model
model_googlenet = GoogleNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
H_googlenet = train_model(model_googlenet, "GoogleNet")
#plot_model_performance(H_googlenet, "GoogleNet")

In [27]:
# Cell for custom plotting
model_name="GoogleNet"
plt.style.use("ggplot")
plt.figure()
N = epochs
#plt.plot(np.arange(0, N), H_googlenet.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H_googlenet.history["val_dense_9_loss"], label="val_loss")
plt.title(f"{model_name} Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss/MAE")
plt.legend(loc="upper right")
plt.savefig(f"{model_name}_loss_plot.png")

plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0, N), H_googlenet.history["dense_13_mae"], label="train_mae")
plt.plot(np.arange(0, N), H_googlenet.history["val_dense_9_mae"], label="val_mae")
plt.title(f"{model_name} Training MAE")
plt.xlabel("Epoch #")
plt.ylabel("MAE")
plt.legend(loc="upper right")
plt.savefig(f"{model_name}_mae_plot.png")

In [ ]:
# build and train VGGNet model
model_vggnet = VGGNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
H_vggnet = train_model(model_vggnet, "VGGNet")
plot_model_performance(H_vggnet, "VGGNet")

In [ ]:
# Initial parameters
epochs = 100
lr = 0.1
batch_size = 64
img_dims_AlexNet = (227, 227, 3)

# Load and preprocess the data
data_AlexNet = []
age_labels = []

# Load image files from the dataset
image_files = [f for f in glob.glob(dataset_path + "/*.jpg", recursive=True)]

print(len(image_files))

random.seed(42)
random.shuffle(image_files)

# Create ground-truth labels from the image paths
for img in image_files:
    image = cv2.imread(img)
    image = cv2.resize(image, (img_dims_AlexNet[0], img_dims_AlexNet[1]))
    image = img_to_array(image)
    data_AlexNet.append(image)

    # Extract age labels from the image paths
    label = int(img.split("/")[-1].split("_")[0])
    age_labels.append(label)

# Pre-process the data
data_AlexNet = np.array(data_AlexNet, dtype="float") / 255.0
labels = np.array(age_labels)


# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data_AlexNet, labels, test_size=0.2, random_state=42)

# Augmenting dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


4521
Data shape: (4521, 227, 227, 3)
Labels shape: (4521,)


In [ ]:
# build and train AlexNet model
model_alexnet = AlexNetModel.build(width=img_dims_AlexNet[0], height=img_dims_AlexNet[1], depth=img_dims_AlexNet[2], classes=1)
H_alexnet = train_model(model_alexnet, "AlexNet")
#plot_model_performance(H_alexnet2, "AlexNet")

In [ ]:
# plot training/validation loss/mae
model_name="AlexNet"

plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0, N), H_alexnet.history["loss"], label="train_loss")
#plt.plot(np.arange(0, N), H_alexnet2.history["val_loss"], label="val_loss")
plt.title(f"{model_name} Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.savefig(f"{model_name}_loss_plot.png")

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H_alexnet.history["mae"], label="train_mae")
#plt.plot(np.arange(0, N), H_alexnet2.history["val_mae"], label="val_mae")
plt.title(f"{model_name} Training Loss and MAE")
plt.xlabel("Epoch #")
plt.ylabel("MAE")
plt.legend(loc="upper right")
plt.savefig(f"{model_name}_mae_plot.png")

In [ ]:
# Initial parameters
epochs = 100
lr = 0.1
batch_size = 64
img_dims_CNN = (200, 200, 3)

# Load and preprocess the data
data_CNN = []
age_labels = []

# Load image files from the dataset
image_files = [f for f in glob.glob(dataset_path + "/*.jpg", recursive=True)]

print(len(image_files))

random.seed(42)
random.shuffle(image_files)

# Create ground-truth labels from the image paths
for img in image_files:
    image = cv2.imread(img)
    image = cv2.resize(image, (img_dims_CNN[0], img_dims_CNN[1]))
    image = img_to_array(image)
    data_CNN.append(image)

    # Extract age labels from the image paths
    label = int(img.split("/")[-1].split("_")[0])
    age_labels.append(label)

# Pre-process the data
data_CNN = np.array(data_CNN, dtype="float") / 255.0
labels = np.array(age_labels)

print("Data shape:", data_CNN.shape)
print("Labels shape:", labels.shape)

# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data_CNN, labels, test_size=0.2, random_state=42)

# Augmenting dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


4521
Data shape: (4521, 200, 200, 3)
Labels shape: (4521,)


In [ ]:
# build and train CNN model
model_cnn = CNNModel.build(width=img_dims_CNN[0], height=img_dims_CNN[1], depth=img_dims_CNN[2], classes=1)
H_cnn = train_model(model_cnn, "CNNModel")
plot_model_performance(H_cnn, "CNNModel")

In [ ]:
# plot training/validation loss/mae
model_name="CNN"

plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0, N), H_cnn.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H_cnn.history["val_loss"], label="val_loss")
plt.title(f"{model_name} Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.savefig(f"{model_name}_loss_plot.png")

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H_cnn.history["mae"], label="train_mae")
plt.plot(np.arange(0, N), H_cnn.history["val_mae"], label="val_mae")
plt.title(f"{model_name} Training Loss and MAE")
plt.xlabel("Epoch #")
plt.ylabel("MAE")
plt.legend(loc="upper right")
plt.savefig(f"{model_name}_mae_plot.png")

In [15]:
# Test the models on a random image
def test_model(model, model_name):
    random_image_path = random.choice(image_files)
    random_image = cv2.imread(random_image_path)
    random_image = cv2.resize(random_image, (img_dims[0], img_dims[1]))

    # Extract the actual age from the image path
    actual_age = int(random_image_path.split("/")[-1].split("_")[0])

    # Preprocess the image for prediction
    random_image = random_image.astype("float") / 255.0
    random_image = img_to_array(random_image)
    random_image = np.expand_dims(random_image, axis=0)

    # Make a prediction using the trained model
    predicted_age = model.predict(random_image)

    # Display the actual age and model's prediction
    print(f"Model: {model_name}")
    print(f"Actual age: {actual_age}")
    print(f"Predicted age: {int(predicted_age[0][0])}\n")

In [ ]:
def test_model_cnn(model, model_name):
    random_image_path = random.choice(image_files)
    random_image = cv2.imread(random_image_path)
    random_image = cv2.resize(random_image, (200, 200))

    # Extract the actual age from the image path
    actual_age = int(random_image_path.split("/")[-1].split("_")[0])

    # Preprocess the image for prediction
    random_image = random_image.astype("float") / 255.0
    random_image = img_to_array(random_image)
    random_image = np.expand_dims(random_image, axis=0)

    # Make a prediction using the trained model
    predicted_age = model.predict(random_image)

    # Display the actual age and model's prediction
    print(f"Model: {model_name}")
    print(f"Actual age: {actual_age}")
    print(f"Predicted age: {int(predicted_age[0][0])}\n")

In [ ]:
def test_model_alexnet(model, model_name):
    random_image_path = random.choice(image_files)
    random_image = cv2.imread(random_image_path)
    random_image = cv2.resize(random_image, (227, 227))

    # Extract the actual age from the image path
    actual_age = int(random_image_path.split("/")[-1].split("_")[0])

    # Preprocess the image for prediction
    random_image = random_image.astype("float") / 255.0
    random_image = img_to_array(random_image)
    random_image = np.expand_dims(random_image, axis=0)

    # Make a prediction using the trained model
    predicted_age = model.predict(random_image)

    # Display the actual age and model's prediction
    print(f"Model: {model_name}")
    print(f"Actual age: {actual_age}")
    print(f"Predicted age: {int(predicted_age[0][0])}\n")

In [ ]:
# Testing ResNet50 model
test_model(model_resnet, "ResNet50")

1/1 [==============================] - 1s 1s/step
Model: ResNet50
Actual age: 34
Predicted age: 29



In [ ]:
# Testing EfficientNetB0 model
test_model(model_effnet, "EfficientNetB0")

1/1 [==============================] - 2s 2s/step
Model: EfficientNetB0
Actual age: 39
Predicted age: 33



In [28]:
# Testing GoogleNet model
test_model(model_googlenet, "GoogleNet")

1/1 [==============================] - 0s 431ms/step
Model: GoogleNet
Actual age: 34
Predicted age: 1



In [ ]:
# Testing AlexNet model
test_model_alexnet(model_alexnet, "AlexNet")

In [ ]:
# Testing VGGNet model
test_model(model_vggnet, "VGGNet")

In [ ]:
# Testing CNN model
test_model_cnn(model_cnn, "CNN")